# CCT College Dublin
 ## Artificial Intelligence and Data Visualisation & Communication

Student: Charles Franklin Jahn 2020315

Lecturer: David Mcquaid & Sam Weiss

BSc (Hons) in Computing in IT - 4nd Year

04/12/2023 - 05/01/2024

In [ ]:
#pip install python-constraint

In [ ]:
import tkinter as tk
from tkinter import ttk
from constraint import Problem, AllDifferentConstraint

In [ ]:
# Create a problem instance
problem = Problem()